# Building a SQL Agent using Langchain
(Using Gemini 2.0)

## Libraries install

In [2]:
!pip install pydantic==2.9.2
!pip install langchain-community
!pip install -U langchain-google-genai
!pip install langchain_google_vertexai
!pip install psycopg2
!pip install --upgrade google-genai
!pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.1
    Uninstalling pydantic_core-2.33.1:
      Successfully uninstalled pydantic_core-2.33.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.3
    Uninstalling pydantic-2.11.3:
      Successfully uninstalled pydantic-2.11.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.9 MB/s eta 0:00:00
  Attempting uninstall: goo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0


In [3]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase

from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

from langchain.agents.agent import (
      AgentExecutor,
      RunnableMultiActionAgent,
)

from langchain.agents import create_tool_calling_agent
from google import genai
import vertexai
from langchain_google_vertexai import ChatVertexAI
from google.colab import auth

## Gemini 2.0 extraction

### Authentication

In [4]:
auth.authenticate_user()

In [5]:
PROJECT_ID = "dashboard-llm-454014"  # Replace with your project ID
REGION = "europe-west1"  # Use the region where your model is available.

# Initialize Vertex AI (using default credentials)
vertexai.init(project=PROJECT_ID, location=REGION)

### Model

In [6]:
vertex_llm = ChatVertexAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=2048,
    max_retries=16,
    stop=None,
    location=REGION)

## Test Database, bicycle, extraction

In [9]:
# Path to your local .db file
db_path = "bicycles.db"

# Create the SQLDatabase object
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [10]:
query = "SELECT * FROM categories LIMIT 3;"  # This lists all tables in the database
db.run(query)

"[(1, 'Children Bicycles'), (2, 'Comfort Bicycles'), (3, 'Cruisers Bicycles')]"

## Configure Database Agent

In [16]:
def configure_database_agent(db: SQLDatabase, llm: ChatVertexAI):
  database_tools = SQLDatabaseToolkit(db=db, llm=llm).get_tools()

  prompt = f"""
  You are an AI SQL agent specializing in generating and executing **syntactically correct SQL queries** in SQLite language.
  Return a final natural language sentence answer given the executed answer of the SQL Query.

  ### **Response Format**
  Your response **must only contain the final natural language sentence**, without any explanation or reasoning.
  """

  messages = [prompt, HumanMessagePromptTemplate.from_template("{input}"), MessagesPlaceholder(variable_name="agent_scratchpad")]

  database_prompt = ChatPromptTemplate.from_messages(messages)

  runnable = create_tool_calling_agent(llm, database_tools, database_prompt)  # type: ignore

  agent = RunnableMultiActionAgent(runnable=runnable,input_keys_arg=["input"], return_keys_arg=["output"])

  database_agent = AgentExecutor(
          name="Database Agent Executor",
          agent=agent,
          tools=database_tools,
          verbose=True,
          max_iterations=40,
          max_execution_time=40,
      )
  print("Database Agent created!")
  return database_agent

In [17]:
database_agent = configure_database_agent(db, vertex_llm)

Database Agent created!


## Running

In [19]:
question = "How many bicycles are sold"
response = database_agent.invoke(question)
print(response['output'])



> Entering new Database Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT sum(quantity) FROM sales WHERE product = 'bicycle'"}`


Error: (sqlite3.OperationalError) no such table: sales
[SQL: SELECT sum(quantity) FROM sales WHERE product = 'bicycle']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Invoking: `sql_db_list_tables` with `{}`


brands, categories, customers, order_items, orders, products, staffs, stocks, stores
Invoking: `sql_db_schema` with `{'table_names': 'order_items, products'}`



CREATE TABLE order_items (
	order_id INTEGER, 
	item_id INTEGER, 
	product_id INTEGER, 
	quantity INTEGER, 
	list_price REAL, 
	discount REAL
)

/*
3 rows from order_items table:
order_id	item_id	product_id	quantity	list_price	discount
1	1	20	1	599.99	0.2
1	2	8	2	1799.99	0.07
1	3	10	2	1549.0	0.05
*/


CREATE TABLE products (
	product_id INTEGER, 
	product_name TEXT, 
	brand_id INTEGER, 
	category_id INTEGER, 
	model_year INTEGER, 
	list_price REAL
)

/*


- No few shots
- No extra tools implemented
- Toolkit kept on default parameters and used all 4 tools within it